In [16]:
import ast
from PIL import Image
from PIL.ExifTags import GPSTAGS
import re
import piexif
import math

Image path input and GSD calculation

In [2]:
image_path = 'zdjecie/DJI_0343.JPG' # Path to input image
# Calculating ground sampling distance
flight_hight = 40 #meters
s_x = 0.000002410000 #meters
focal_length = 0.01026000 #meters
gsd = (flight_hight*s_x)/focal_length #meters

print(gsd)

0.009395711500974657


Getting coordinates of input image center

In [3]:
def get_image_metadata(image_path):
    """Extracts image width, height, and GPS coordinates from EXIF metadata."""
    img = Image.open(image_path)
    width, height = img.size  # Get original dimensions
    
    exif_data = piexif.load(img.info.get("exif", b""))  # Load EXIF data

    gps_info = exif_data.get("GPS", {})
    gps_data = {}

    if gps_info:
        for key, val in gps_info.items():
            tag = GPSTAGS.get(key, key)
            gps_data[tag] = val

    # Convert GPS DMS format to Decimal Degrees
    def convert_to_degrees(value):
        """Convert EXIF GPS coordinates to decimal degrees."""
        d = value[0][0] / value[0][1]  # Degrees
        m = value[1][0] / value[1][1]  # Minutes
        s = value[2][0] / value[2][1]  # Seconds
        return d + (m / 60.0) + (s / 3600.0)

    if "GPSLatitude" in gps_data and "GPSLongitude" in gps_data:
        lat = convert_to_degrees(gps_data["GPSLatitude"])
        lon = convert_to_degrees(gps_data["GPSLongitude"])

        if gps_data["GPSLatitudeRef"] == b'S':  # South is negative
            lat = -lat
        if gps_data["GPSLongitudeRef"] == b'W':  # West is negative
            lon = -lon
    else:
        lat, lon = None, None

    return width, height, lat, lon


orig_w, orig_h, lat, lon = get_image_metadata(image_path)
print(f"Original Size: {orig_w}x{orig_h}, GPS: ({lat}, {lon})")


Original Size: 5472x3648, GPS: (54.13496063888889, 19.93641866666667)


Resizing image to fit into model

In [4]:
def resize_image(image_path, output_size=(2048, 1365), save_path="resized_image.jpg"):
    """Resizes the image to 2048x1365 using LANCZOS interpolation and saves it."""
    img = Image.open(image_path)
    img_resized = img.resize(output_size, Image.LANCZOS)  # Use LANCZOS instead of ANTIALIAS
    
    return img_resized

Centroid transformation to ground coordinates

In [5]:
def pixel_to_ground_coordinates(gsd, center_lat, center_lon, old_w, old_h, new_w, new_h, xx, yy):
    """
    Converts pixel coordinates (xx, yy) to geographic coordinates (latitude, longitude).
    """
    # Calculate scale factor
    scale_x = orig_w / new_w
    scale_y = orig_h / new_h

    # Convert back to original scale
    orig_cx = xx * scale_x
    orig_cy = yy * scale_y

    # Compute pixel offset from the image center
    delta_x = (orig_cx - old_w / 2) * gsd
    delta_y = (orig_cy - old_h / 2) * gsd

    # Convert pixel offsets to degrees
    earth_radius = 6378137.0  # Earth's radius in meters
    lat_per_meter = 1 / (earth_radius * (math.pi / 180))  # Meters to degrees latitude
    lon_per_meter = lat_per_meter / math.cos(math.radians(center_lat))  # Meters to degrees longitude

    # Compute new latitude and longitude
    new_lat = center_lat - delta_y * lat_per_meter  # Negative delta_y moves north
    new_lon = center_lon + delta_x * lon_per_meter  # Positive delta_x moves east

    return new_lat, new_lon

Loading model output data and computing centroids

In [6]:
def extract_data_from_output(data):
    """
    Extracts bounding box coordinates and class IDs dynamically, 
    ensuring correct order even if the CSV format changes.
    """
    if not data or 'output' not in data[0]:
        print("❌ Error: Invalid input data format")
        return None

    # Extract CSV content (skip the header)
    lines = data[0]['output'].strip().split("\n")

    if len(lines) < 2:
        print("❌ Error: CSV data is missing values")
        return None

    header = lines[0].split(",")  # Extract column names
    csv_content = lines[1]  # Extract actual values
    
    # Debug: Print detected header
    print(f"🛠 Detected Header: {header}")

    # Use regex to extract list-like substrings
    matches = re.findall(r'\[.*?\]', csv_content)

    if len(matches) != len(header) - 1:
        print("❌ Error: Mismatch in detected values and header length")
        print(f"🔍 Extracted Matches: {matches}")
        return None

    # Dictionary to store extracted values dynamically
    extracted_values = {}

    # Map extracted values to column names dynamically
    for col_name, match in zip(header, matches):
        extracted_values[col_name.strip()] = ast.literal_eval(match)

    # Extract values safely
    try:
        y_max = extracted_values.get("y_max", [])
        y_min = extracted_values.get("y_min", [])
        x_max = extracted_values.get("x_max", [])
        x_min = extracted_values.get("x_min", [])
        class_ids = extracted_values.get("Class ID", [])

        # Ensure class IDs are integers
        class_ids = [int(cid) for cid in class_ids]

        # Compute centroids
        centroids = [((float(xmax) + float(xmin)) / 2, (float(ymax) + float(ymin)) / 2) 
                     for xmax, xmin, ymax, ymin in zip(x_max, x_min, y_max, y_min)]

        # Debug: Print extracted values
        print(f"✅ Extracted Class IDs: {class_ids}")
        print(f"✅ Computed Centroids: {centroids}")

        return centroids, class_ids

    except Exception as e:
        print(f"❌ Error while processing data: {e}")
        return None

Saving centroids to CSV

In [7]:
import csv

def save_centroids_to_csv(centroids, class_ids, output_csv="centroids.csv"):
    """Saves centroids (lat, lon) and class IDs to a CSV file for QGIS."""
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Latitude", "Longitude", "Class ID"])  # CSV header
        
        for (lat, lon), class_id in zip(centroids, class_ids):
            writer.writerow([lat, lon, class_id])

    print(f"Centroids saved to {output_csv}")

Getting corner coordinates of input image

In [9]:
def get_image_corners(center_lat, center_lon, gsd, img_width, img_height):
    """
    Computes the GPS coordinates of the four corners of an image.
    """

    # Compute pixel shifts from center
    half_w = (img_width / 2) * gsd  # Half width in meters
    half_h = (img_height / 2) * gsd  # Half height in meters

    # Convert meters to degrees
    earth_radius = 6378137.0  # Radius of Earth in meters
    lat_per_meter = 1 / (earth_radius * (3.141592653589793 / 180))  # Degrees per meter
    lon_per_meter = lat_per_meter / math.cos(math.radians(center_lat))  # Adjust for latitude

    # Calculate corner coordinates
    top_left = (center_lat + half_h * lat_per_meter, center_lon - half_w * lon_per_meter)
    top_right = (center_lat + half_h * lat_per_meter, center_lon + half_w * lon_per_meter)
    bottom_left = (center_lat - half_h * lat_per_meter, center_lon - half_w * lon_per_meter)
    bottom_right = (center_lat - half_h * lat_per_meter, center_lon + half_w * lon_per_meter)

    return {
        "top_left": top_left,
        "top_right": top_right,
        "bottom_left": bottom_left,
        "bottom_right": bottom_right
    }

Execution of defined functions

In [10]:
orig_w, orig_h, lat, lon = get_image_metadata(image_path)
print(f"Original Size: {orig_w}x{orig_h}, GPS: ({lat}, {lon})")

Original Size: 5472x3648, GPS: (54.13496063888889, 19.93641866666667)


In [11]:
resized_image = resize_image(image_path)
#resized_image.show()  # Show resized image

In [12]:
# Connecting to model
from inference_sdk import InferenceHTTPClient

client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="your_APIKEY" #Can't show mine
)

result = client.run_workflow(
    workspace_name="praca-7cas9",
    workflow_id="custom-workflow-2",
    images={
        "image": resized_image # your input image, might need resizing
    },
    use_cache=False # cache workflow definition for 15 minutes
)

In [13]:
print(result)
data=result

[{'output': 'x_max,Class ID,x_min,y_min,y_max,timestamp\n"[1937.0, 2045.0, 2027.0, 1349.0, 1660.0, 524.0, 919.0, 1866.0, 114.0, 1931.0, 1659.0, 1732.0, 1555.0, 1632.0, 1948.0, 1304.0, 1566.0, 129.0, 1431.0, 1482.0, 1353.0, 1590.0]","[\'3\', \'1\', \'2\', \'1\', \'1\', \'1\', \'3\', \'0\', \'2\', \'2\', \'1\', \'2\', \'3\', \'2\', \'2\', \'2\', \'2\', \'2\', \'2\', \'2\', \'2\', \'2\']","[1786.0, 1879.0, 1958.0, 1287.0, 1557.0, 432.0, 788.0, 1559.0, 12.0, 1900.0, 1556.0, 1681.0, 1493.0, 1609.0, 1811.0, 1280.0, 1541.0, 0.0, 1407.0, 1457.0, 1324.0, 1561.0]","[148.0, 1086.0, 1234.0, 986.0, 462.0, 0.0, 402.0, 252.0, 787.0, 1283.0, 820.0, 1281.0, 0.0, 1096.0, 1269.0, 572.0, 1192.0, 755.0, 675.0, 1087.0, 682.0, 1349.0]","[297.0, 1265.0, 1296.0, 1063.0, 562.0, 79.0, 474.0, 502.0, 840.0, 1306.0, 886.0, 1317.0, 39.0, 1123.0, 1365.0, 595.0, 1212.0, 992.0, 707.0, 1110.0, 703.0, 1364.0]",2025-02-03T21:49:50.535372+00:00\n'}]


In [17]:
centroids, class_ids = extract_data_from_output(data)

🛠 Detected Header: ['x_max', 'Class ID', 'x_min', 'y_min', 'y_max', 'timestamp']
✅ Extracted Class IDs: [3, 1, 2, 1, 1, 1, 3, 0, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✅ Computed Centroids: [(1861.5, 222.5), (1962.0, 1175.5), (1992.5, 1265.0), (1318.0, 1024.5), (1608.5, 512.0), (478.0, 39.5), (853.5, 438.0), (1712.5, 377.0), (63.0, 813.5), (1915.5, 1294.5), (1607.5, 853.0), (1706.5, 1299.0), (1524.0, 19.5), (1620.5, 1109.5), (1879.5, 1317.0), (1292.0, 583.5), (1553.5, 1202.0), (64.5, 873.5), (1419.0, 691.0), (1469.5, 1098.5), (1338.5, 692.5), (1575.5, 1356.5)]


In [18]:
real_world_coords = []
for cx, cy in centroids:
    new_lat, new_lon = pixel_to_ground_coordinates(gsd, lat, lon, orig_w, orig_h, 2048, 1365, cx, cy)
    real_world_coords.append((new_lat, new_lon))  # Append the tuple correctly

# Print results
for idx, (lat, lon) in enumerate(real_world_coords):
    print(f"Centroid {idx + 1}: Latitude {lat}, Longitude {lon}")

Centroid 1: Latitude 54.135064400921834, Longitude 19.9367410349008
Centroid 2: Latitude 54.13484943305794, Longitude 19.936779719088896
Centroid 3: Latitude 54.134829244575435, Longitude 19.936791459066377
Centroid 4: Latitude 54.134883494073094, Longitude 19.936531832351246
Centroid 5: Latitude 54.13499909851197, Longitude 19.936643650825296
Centroid 6: Latitude 54.135105680165374, Longitude 19.93620850182388
Centroid 7: Latitude 54.13501579066509, Longitude 19.93635303826796
Centroid 8: Latitude 54.135029550412945, Longitude 19.936683682223922
Centroid 9: Latitude 54.134931089266466, Longitude 19.93604876114667
Centroid 10: Latitude 54.13482259027115, Longitude 19.9367618204347
Centroid 11: Latitude 54.134922179265814, Longitude 19.936643265908
Centroid 12: Latitude 54.13482157520779, Longitude 19.936681372720155
Centroid 13: Latitude 54.13511019155811, Longitude 19.93661112531391
Centroid 14: Latitude 54.13486432065397, Longitude 19.93664826983283
Centroid 15: Latitude 54.134817514

In [26]:
save_centroids_to_csv(real_world_coords, class_ids)

Centroids saved to centroids3.csv


In [28]:
gps_corners = get_image_corners(lat, lon, gsd, orig_w, orig_h)

# Export to CSV
csv_filename = "image_corners.csv"
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Corner", "Latitude", "Longitude"])
    for corner, (lat, lon) in gps_corners.items():
        writer.writerow([corner, lat, lon])

print(f"GPS corners saved to {csv_filename}")

GPS corners saved to image_corners1.csv
